# This notebook, I will work on following:
* Import the News urls using Finnhub API. 
* Chat with the news. 

In [1]:
from financial_data import FinancialDataFinnHub
import os

In [2]:
# Create Finnhub object
symbol = "AMD"
api_key = os.environ.get("FINHUB_API_KEY")
nvda = FinancialDataFinnHub(symbol, api_key)

## Load News then chat about news:
* Use `FinancialDataFinnHub`class to load news url
* langchain `UnstructuredURLLoader` to load news
* langchain `RecursiveCharacterTextSplitter` to split the data
* Remove unreachable news from the list
* Create `OpenAIEmbeddings` model
* Split text with `RecursiveCharacterTextSplitter`
* Store vectors in `Chroma`
* Create prompt template with `PromprTemplate`
* Create llm chain and chat with News. Look `basics.py`and `index_vectorstore_index_creation.ipynb`


In [3]:
# Use FinancialHub to load news url
news_urls = nvda.company_news['url']
urls = [news_urls.iloc[i][0] for i in range(len(news_urls))]

In [4]:
# langchain UnstructuredURLLoader to load news
from langchain.document_loaders import SeleniumURLLoader

chrome_driver_path = r"C:\Development\chromedriver.exe"

#browser = webdriver.Chrome(executable_path=r"C:\path\to\chromedriver.exe")
loader = SeleniumURLLoader(urls=urls, executable_path=chrome_driver_path)
data = loader.load()

In [5]:
data

[Document(page_content='Home\n\nEarnings Analysis\n\nAMD Q1 Preview: 4 Things I\'ll Focus On\n\nApr. 26, 2023 1:40 PM ET\n\nAdvanced Micro Devices, Inc. (AMD)\n\nINTC, NVDA\n\n5 Comments\n\n2 Likes\n\nEnvision Research\n\nInvesting Group Leader\n\nFollow\n\nSummary\n\nAdvanced Micro Devices is scheduled to report earnings on May 2 post-market for its 2023 FY Q1.\n\nI expect to see signs of the end of margin contraction and also asset utilization contraction.\n\nI expect to see a strong balance sheet and the continuation of share repurchases.\n\nThese signs will provide insights not only into AMD’s own business but also the overall status of the cycle in the chip sector.\n\nI will listen closely to management\'s comments and data that either confirm or contradict my above expectations.\n\nI do much more than just articles at Envision Early Retirement: Members get access to model portfolios, regular updates, a chat room, and more.  Learn More »\n\nDavid Becker\n\nThesis\n\nAdvanced Micro

In [6]:
# Remove news if the content is not reacheable. 
for i, d in enumerate(data):
    p_content = d.page_content
    if p_content == '':
        data.pop(i)
print(f"You got {len(data)} news to read about {symbol}. ")

You got 13 news to read about AMD. 


In [7]:
# Create `OpenAIEmbeddings` model
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [8]:
# Split text with RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data)

In [9]:
docs

[Document(page_content="Home\n\nEarnings Analysis\n\nAMD Q1 Preview: 4 Things I'll Focus On\n\nApr. 26, 2023 1:40 PM ET\n\nAdvanced Micro Devices, Inc. (AMD)\n\nINTC, NVDA\n\n5 Comments\n\n2 Likes\n\nEnvision Research\n\nInvesting Group Leader\n\nFollow\n\nSummary\n\nAdvanced Micro Devices is scheduled to report earnings on May 2 post-market for its 2023 FY Q1.\n\nI expect to see signs of the end of margin contraction and also asset utilization contraction.\n\nI expect to see a strong balance sheet and the continuation of share repurchases.\n\nThese signs will provide insights not only into AMD’s own business but also the overall status of the cycle in the chip sector.\n\nI will listen closely to management's comments and data that either confirm or contradict my above expectations.\n\nI do much more than just articles at Envision Early Retirement: Members get access to model portfolios, regular updates, a chat room, and more.  Learn More »\n\nDavid Becker\n\nThesis", metadata={'source

In [10]:
# Store vectors in Chroma 
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)

Using embedded DuckDB without persistence: data will be transient


#### Create prompt template and add to the chain. 

# ConversationalRetrievalChain:
https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html?highlight=ConversationalRetrievalChain#conversationalretrievalchain-with-search-distance

https://python.langchain.com/en/latest/reference/modules/chains.html?highlight=ConversationalRetrievalChain#langchain.chains.ConversationalRetrievalChain

Check Mayo's code makechain.ts. Bottom url will show how it works in python

In [13]:
# Create prompt template with PromprTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationEntityMemory

chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
memory = ConversationEntityMemory(llm=chat)
cd_template = """Given the following conversation and a follow up question, rephras the follow up question to be a standalone
question. 

Chat History:
{chat_history}
Follow Up Input: {question}
standalone question:"""
CONDENSE_PROMPT = PromptTemplate(template=cd_template, 
                                           input_variables=["chat_history", "question"])

qa_prompt = """You are AI Entertainer and brilliant in economics and finance. Please use the following pieces of context 
to answerthe question at the end. You must make some kind of conclusion and DO NOT give me a answer that YOU DON'T KNOW. 
Your answer is NOT the advise and the porpose is for your opinion is entertainment use.

{context}
Question: {question}
Your Answer: 
"""

QA_PROMPT = PromptTemplate(template=qa_prompt, 
                                    input_variables=['context', 'question'])

# Create llm chain

retriever_chain = ConversationalRetrievalChain.from_llm(chat, retriever=vectorstore.as_retriever(),
                                                        condense_question_prompt=CONDENSE_PROMPT,
                                                        verbose=True, 
                                                         return_source_documents=True)

In [16]:
from langchain.chains import RetrievalQA

retrieval_qa = RetrievalQA.from_chain_type(llm=chat, chain_type='stuff',
                                         retriever=vectorstore.as_retriever(),
                                         )

## Additional feature:

Add serch tool to search extra information about the keyward from the article

In [17]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, Tool, initialize_agent, tool
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

In [18]:
search = SerpAPIWrapper()
python_repl = PythonREPL()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. when I tell you 'use search', then use \
        this tool."
    ),
    Tool(
        name='pythonREPL',
        func=python_repl.run,
        description="useful for when you need to run python code to find the answer. When I tell you to 'use python', then use \
        this tool"
    ),
    Tool(
        name='news qa system',
        func=retrieval_qa.run,
        description="useful when you need to answer questions about ther recent news. when I tell you to 'search news', then use \
        this tool"
    )
]

In [19]:
tools

[Tool(name='Search', description="useful for when you need to answer questions about current events. when I tell you 'use search', then use         this tool.", args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x00000238E1C80760>, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='6e4843553bd0a09d09846bcb92928fbc9c79942ae376a4783b11af8944ba3c64', aiosession=None)>, coroutine=None),
 Tool(name='pythonREPL', description="useful for when you need to run python code to find the answer. When I tell you to 'use python', then use         this tool", args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x00000238E1C80760>, func=<bound method PythonREPL.run of PythonREPL(glo

## Custom memory chain:

In [20]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(temperature=0)

In [21]:
# Here it is by default set to "AI"
conversation_memory = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [22]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [23]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, memory=memory, verbose=True)

In [24]:
agent.run("use news to tell me about recent AMD news using the tools.")



> Entering new AgentExecutor chain...
I should use the news qa system to find recent AMD news.
Action: search news
Action Input: "AMD recent news"
Observation: search news is not a valid tool, try another one.
Thought:I should use the news qa system to find recent AMD news.
Action: news qa system
Action Input: "What is the latest news about AMD?"
Observation: The latest news about AMD is that the company is scheduled to release its earnings report for its FY Q1 ending March 2023 on May 2, 2023, post-market. AMD has been facing strong headwinds in the past few quarters due to a supply glut and margin compression. However, the company has been adding new and improved products across its segments, which means that the market share gains are likely to continue in the coming years. Investors should take a closer look at the company before joining the AMD bandwagon ahead of its Q1 report.
Thought:I now know the recent news about AMD.
Final Answer: The latest news about AMD is that the comp

'The latest news about AMD is that the company is scheduled to release its earnings report for its FY Q1 ending March 2023 on May 2, 2023, post-market.'

In [25]:
agent.run("use news qa system to tell me all the headlines about the semiconductor companies. ")



> Entering new AgentExecutor chain...
I need to search for a news qa system that can provide me with headlines about semiconductor companies.
Action: Search
Action Input: "news qa system for semiconductor company headlines"
Observation: Semiconductor Industry News - April 2023 Update ... and automotive OEMs are unwilling to redesign existing systems to implement ...
Thought:This looks like a promising news qa system for semiconductor company headlines.
Action: news qa system
Action Input: "semiconductor company headlines"
Observation: One recent headline related to a semiconductor company is "ASML's Leadership Position in Semiconductor Lithography Market Ensures Its Continued Importance." Another headline is "Biden Administration Strikes Deal with Dutch Officials to Prohibit Sales of Some Immersion Lithography Gear to China."
Thought:I have found multiple recent headlines about semiconductor companies.
Final Answer: Some recent headlines related to semiconductor companies include "AS

'Some recent headlines related to semiconductor companies include "ASML\'s Leadership Position in Semiconductor Lithography Market Ensures Its Continued Importance" and "Biden Administration Strikes Deal with Dutch Officials to Prohibit Sales of Some Immersion Lithography Gear to China."'

In [26]:
agent.run("Can you return the list of headlines about semiconductor companies using New qa system.")



> Entering new AgentExecutor chain...
I need to use the news qa system to search for headlines about semiconductor companies.
Action: Search news
Action Input: "semiconductor companies"
Observation: Search news is not a valid tool, try another one.
Thought:I need to use a different tool to search for headlines about semiconductor companies.
Action: Search
Action Input: "semiconductor companies headlines"
Observation: Get the latest breaking news and top news headlines at Reuters.com. ... Taiwan Semiconductor Manufacturing Co (TSMC) as part of steps the company was taking ...
Thought:I need to refine my search to get more specific results.
Action: Search
Action Input: "semiconductor companies latest headlines"
Observation: WASHINGTON—April 25, 2023—The Semiconductor Industry Association (SIA) today released the following statement from SIA President and CEO John Neuffer ...
Thought:I need to look through the search results to find the list of headlines.
Action: Search
Action Input: "l

'The list of semiconductor companies in the news includes II-VI Incorporated, Advantest, Moov Technologies, AMD, Apple, Arrow Electronics, Inc., Applied Materials, and Qualcomm.'

## ConversationalRetrieverChain

In [12]:
# Chat with News
chat_history = []
question = "Please provide hedlines related to semiconductor company."
result1 = chain({"question": question, "chat_history": chat_history})

In [13]:
result1

{'question': 'Please provide hedlines related to semiconductor company.',
 'chat_history': [],
 'answer': "1. The Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n2. These 3 Hot Tech Names Could Be Trillion-Dollar Stocks by 2030\n3. 1 Flashing Warning Sign in Nvidia's Stock That You Shouldn't Ignore\n4. Jason Seidl's top recommendation has been Daseke (NASDAQ:DSKE)\n5. Quinn Bolton's best rating in the past decade has been on ACM Research (NASDAQ:ACMR)",
 'source_documents': [Document(page_content='Motley Fool\n\nThe Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n\nHarsh Chauhan, The Motley Fool\n\n4 min read\n\nNvidia (NASDAQ: NVDA) stock has been on fire over the past six months. Frank Lee of banking and financial services giant HSBC was the last one among the 48 analysts covering Nvidia with a negative rating on the stock. Lee wrote in a research note that he is "shocked" by Nvidia\'s pricing power in artificial intelligence (AI) chips.\n\nContinue re

In [14]:
chat_1 = result1['question'], result1['answer']
chat_history = [chat_1]

In [15]:
chat_history

[('Please provide hedlines related to semiconductor company.',
  "1. The Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n2. These 3 Hot Tech Names Could Be Trillion-Dollar Stocks by 2030\n3. 1 Flashing Warning Sign in Nvidia's Stock That You Shouldn't Ignore\n4. Jason Seidl's top recommendation has been Daseke (NASDAQ:DSKE)\n5. Quinn Bolton's best rating in the past decade has been on ACM Research (NASDAQ:ACMR)")]

In [16]:
question = "Give me a reason why NVDA Could Keep Skyrocketing"
result2 = chain({"question": question, "chat_history": chat_history})

In [17]:
result2['answer']

"According to the Motley Fool article, the single biggest reason why Nvidia stock could keep skyrocketing is its pricing power in artificial intelligence (AI) chips, as noted by Frank Lee of HSBC. However, it is important to note that another Motley Fool article points out a flashing warning sign in Nvidia's stock, as the company has posted negative revenue growth in the past two quarters. While there is potential for Nvidia to continue to perform well in the future, investors should also consider potential risks and do their own research before making any investment decisions."

In [18]:
result2['answer']

"According to the Motley Fool article, the single biggest reason why Nvidia stock could keep skyrocketing is its pricing power in artificial intelligence (AI) chips, as noted by Frank Lee of HSBC. However, it is important to note that another Motley Fool article points out a flashing warning sign in Nvidia's stock, as the company has posted negative revenue growth in the past two quarters. While there is potential for Nvidia to continue to perform well in the future, investors should also consider potential risks and do their own research before making any investment decisions."

In [19]:
chat2 =  result2['question'], result2['answer']
chat_history.append(chat2)

In [20]:
len(chat_history)

2

In [21]:
question = "Do you have information of NVDA warning sign?"
result3 = chain({"question": question, "chat_history": chat_history})

In [22]:
result3

{'question': 'Do you have information of NVDA warning sign?',
 'chat_history': [('Please provide hedlines related to semiconductor company.',
   "1. The Single Biggest Reason Why Nvidia Stock Could Keep Skyrocketing\n2. These 3 Hot Tech Names Could Be Trillion-Dollar Stocks by 2030\n3. 1 Flashing Warning Sign in Nvidia's Stock That You Shouldn't Ignore\n4. Jason Seidl's top recommendation has been Daseke (NASDAQ:DSKE)\n5. Quinn Bolton's best rating in the past decade has been on ACM Research (NASDAQ:ACMR)"),
  ('Give me a reason why NVDA Could Keep Skyrocketing',
   "According to the Motley Fool article, the single biggest reason why Nvidia stock could keep skyrocketing is its pricing power in artificial intelligence (AI) chips, as noted by Frank Lee of HSBC. However, it is important to note that another Motley Fool article points out a flashing warning sign in Nvidia's stock, as the company has posted negative revenue growth in the past two quarters. While there is potential for Nvidi

In [33]:
result3['chat_history']

[('Please provide hedlines related to semiconductor company.',
  "NVDA among Wall Street's top 10 stock calls this week. ISM Liquidity Bets includes NVDA in their analysis."),
 ("Can you tell me more about Taiwan Semiconductor Q1'23 Review?",
  "There is no information provided in the given context about the review for Taiwan Semiconductor in Q1'23.")]

In [34]:
result3['answer']

"Based on the news provided, semiconductor companies seem to be facing some challenges, with Nvidia experiencing a 2.96% fall and worries over shrinking margins affecting most investors. However, it's important to note that there are still favorable catalysts in place for growth stocks like PI and QCOM. Overall, it's a mixed picture for semiconductor companies and investors should do their own research before making any decisions."

In [36]:
chat3 = result3['question'], result3['answer']
chat_history.append(chat3)

In [37]:
question = "what was the first question I asked? You should find inside the chat_history."
result4 = chain({"question": question, "chat_history": chat_history})

In [38]:
result4['answer']

'As an AI language model, I do not have personal opinions. However, based on the given news articles, it seems that there are concerns about shrinking margins in the technology sector, including semiconductor companies like Nvidia. The article from Supply Chain Values suggests that the tug of war between bulls and bears in the market would continue unless certain levels are convincingly broken. It is important for investors to be cautious and consider the potential impact of any monetary policy changes on the deteriorating economic and market environment.'

In [39]:
chat_history

[('Please provide hedlines related to semiconductor company.',
  "NVDA among Wall Street's top 10 stock calls this week. ISM Liquidity Bets includes NVDA in their analysis."),
 ("Can you tell me more about Taiwan Semiconductor Q1'23 Review?",
  "There is no information provided in the given context about the review for Taiwan Semiconductor in Q1'23."),
 ('Please tell me your opinion about semiconductor companies based on the news.',
  "Based on the news provided, semiconductor companies seem to be facing some challenges, with Nvidia experiencing a 2.96% fall and worries over shrinking margins affecting most investors. However, it's important to note that there are still favorable catalysts in place for growth stocks like PI and QCOM. Overall, it's a mixed picture for semiconductor companies and investors should do their own research before making any decisions.")]

## pinecone webiner code:

https://colab.research.google.com/drive/1ZbXzzMFyMQ-nQf-gQXWAv4IRDI90rXpa?usp=sharing&_hsmi=255277680